In [1]:
suppressPackageStartupMessages({
    library(Seurat)
    library(harmony)
    library(stringr)
    library(ggplot2)
    library(tidyr)
    library(Matrix)
    library(Azimuth)
    library(dplyr)
    library(RColorBrewer)
    library(UCell)
    library(Nebulosa)
    library(SeuratDisk)
    library(ggsci)
    library(DoubletFinder)
    library(scRepertoire)    
    library(scCustomize)
    library(scater)
    library(ggpubr)
    library(ggbeeswarm)
    library(UpSetR)
    library(SCP)

    source('helper_functions.R')
})

set.seed(123)

In [2]:
prepost <- readRDS("../ProcessedData/seurat/clustered_ip4.rds")

In [32]:
table(prepost$sample_id)


 P3IP  P4IP  P6IP  P8IP 
 6797  5574  4002 20452 

In [2]:
samples <- c(
    'P4IP','P6IP','P8IP', 'P3IP'
)

contig_list <- list()

for (sample in samples) {
    contig.path = Sys.glob(paste0("../RawData/cellranger_out/",sample,"/*/*/*/filtered_contig_annotations.csv"))
    contig_list[[sample]] <- read.csv(contig.path)
    if (length(contig_list[[sample]]) == 32 && colnames(contig_list[[sample]])[[1]] == 'sample') {
        contig_list[[sample]] = contig_list[[sample]][2:32] 
    }
}

In [3]:
combined.TCR <- combineTCR(
    contig_list, samples = names(contig_list), removeNA = TRUE, removeMulti = TRUE, filterMulti = TRUE, filterNonproductive = TRUE
)

In [110]:
total_first_dim <- sum(sapply(combined.TCR, function(x) dim(x)[1]))
print(paste0("Number of T Cells pre filtering: ", total_first_dim))

[1] "Number of T Cells pre filtering: 47402"


In [111]:
# Function to swap parts of the barcode
swap_barcode <- function(barcode) {
  parts <- strsplit(barcode, "_")[[1]]
  if (parts[1] == 'P8IP') { parts[1] <- paste0(parts[1],'_04')}
  paste(parts[2], parts[1], sep = "_")
}

# Apply the swap to all dataframes in combined.TCR
combined.TCR <- lapply(combined.TCR, function(df) {
  df$barcode <- sapply(df$barcode, swap_barcode)
  return(df)
})

In [112]:
Patient <- sub("^(P\\d+).*", "\\1", names(combined.TCR))
Day <- rep('IP', length(names(combined.TCR)))

In [113]:
combined.TCR <- addVariable(combined.TCR,  variable.name = "Day",  variables = Day)
combined.TCR <- addVariable(combined.TCR,  variable.name = "Patient",  variables = Patient)

In [114]:
prepost <- combineExpression(combined.TCR, prepost, cloneCall="aa", 
    group.by = "sample", proportion = FALSE, 
    cloneSize=c(Single=1, Small=5, Medium=20, Large=100, Hyperexpanded=500)
)

In [118]:
saveRDS(prepost, "../ProcessedData/seurat/clustered_ip4.rds")